In [1]:
# Load modules
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import numpy as np
import pandas as pd
import pickle

from train_ann_functions import AsuSharedNN, ll, normaliser_shared

In [2]:
# Load data
data_train = pd.read_csv('data/swissmetro_clean_train.csv')
data_test = pd.read_csv('data/swissmetro_clean_test.csv')

In [3]:
# Set scalars
J = 3
K = 2

# Set variables
Xvars = ['TRAIN_COST','TRAIN_TT','TRAIN_HE','SM_COST','SM_TT','SM_HE','CAR_COST','CAR_TT']
X_train = data_train[Xvars]
X_test = data_test[Xvars]
X = pd.concat([X_train,X_test],axis=0)

transformer = normaliser_shared(X_shared = [1,0,0,1,0,0,1,0],shared_locations = [[0,3,6]])
transformer.fit(X)

X_train = transformer.transform(X_train)
X_test = transformer.transform(X_test)
X = transformer.transform(X)

y_train = data_train['CHOICE'].to_numpy() - 1
y_test = data_test['CHOICE'].to_numpy() - 1
y = np.r_[y_train,y_test]

In [4]:
R = 100

ll_full_list = []
ll_train_list = []
ll_test_list = []
r2_test_list = []
mg_U = []
V = []
p = []

for r in range(R):
    # Set model
    model = AsuSharedNN(topology=(5,5),activation='tanh',output_bias=True,from_logits=True)
    model.fit(X_train,y_train,[1,1,1,2,2,2,3,3],[1,0,0,1,0,0,1,0],early_stopping=True,validation_split=0.1)

    # Get log-likelihood
    ll_full = ll(y,model.predict_proba(X))
    ll_train = ll(y_train,model.predict_proba(X_train))
    ll_test = ll(y_test,model.predict_proba(X_test))
    r2 = 1 - ll_test/(len(X_test)*np.log(1/J))

    ll_full_list.append(ll_full)
    ll_train_list.append(ll_train)
    ll_test_list.append(ll_test)
    r2_test_list.append(r2)

    print(str(r+1) + '/' + str(R) + ' / Log-lik (full): ' + str(round(ll_full,2)) + ' / LL (train) = ' + str(round(ll_train,2)) + ' / LL (test) = ' + str(round(ll_test,2)) + ' / Rho-sq (test): ' + str(round(r2,4)))

    mg_U.append(model.gradient(X_test,transformer))
    V.append(model.predict_utility(X_test))
    p.append(model.predict_proba(X_test))

1/100 / Log-lik (full): -6946.72 / LL (train) = -5552.56 / LL (test) = -1394.15 / Rho-sq (test): 0.2981
2/100 / Log-lik (full): -6942.79 / LL (train) = -5549.43 / LL (test) = -1393.36 / Rho-sq (test): 0.2985
3/100 / Log-lik (full): -6940.99 / LL (train) = -5548.24 / LL (test) = -1392.75 / Rho-sq (test): 0.2988
4/100 / Log-lik (full): -6941.75 / LL (train) = -5549.87 / LL (test) = -1391.88 / Rho-sq (test): 0.2993
5/100 / Log-lik (full): -6935.95 / LL (train) = -5544.12 / LL (test) = -1391.83 / Rho-sq (test): 0.2993
6/100 / Log-lik (full): -6939.4 / LL (train) = -5545.0 / LL (test) = -1394.4 / Rho-sq (test): 0.298
7/100 / Log-lik (full): -6937.07 / LL (train) = -5546.04 / LL (test) = -1391.03 / Rho-sq (test): 0.2997
8/100 / Log-lik (full): -6939.29 / LL (train) = -5547.03 / LL (test) = -1392.26 / Rho-sq (test): 0.2991
9/100 / Log-lik (full): -6946.62 / LL (train) = -5553.37 / LL (test) = -1393.25 / Rho-sq (test): 0.2986
10/100 / Log-lik (full): -6945.05 / LL (train) = -5551.26 / LL (test

In [5]:
# Stack metrics
ll_full_array = np.array(ll_full_list)
ll_train_array = np.array(ll_train_list)
ll_test_array = np.array(ll_test_list)
r2_test_array = np.array(r2_test_list)

# Create series
metrics = pd.DataFrame(np.c_[ll_full_array,ll_train_array,ll_test_array,r2_test_array],columns=['Log-lik (full)','Log-lik (train)','Log-lik (test)','Rho-sq (test)'])
metrics.to_csv('results/asushared_swissmetro_metrics.csv')
metrics.mean()

Log-lik (full)    -6941.322295
Log-lik (train)   -5548.834829
Log-lik (test)    -1392.487466
Rho-sq (test)         0.298951
dtype: float64

In [6]:
# Save pickle file
with open('results/asushared_swissmetro.pickle', 'wb') as handle:
    pickle.dump([mg_U,V,p], handle, protocol=pickle.HIGHEST_PROTOCOL)